
# Day 12 · Notebook 4 — Cross‑Model Interoperability  (Concept #100)

We will run the *same* prompt on three models—`gpt-4o-mini`, `meta-llama-3-8b-instruct`, and `mistralai/Mixtral‑7B`—then post‑process to a common JSON schema.

> **Why?** Moving workloads between providers avoids vendor lock‑in and enables graceful degradation when one API rate‑limits.


In [ ]:

# NOTE: This demo uses the openai and together.ai endpoints as examples only.
import json, os, openai, requests

prompt = "Extract the top three key takeaways (max 12 words each) from the text: 'Prompt engineering boosts LLM reliability.' Return JSON list."

def gpt_call():
    return openai.ChatCompletion.create(
        model="gpt-4o-mini",
        messages=[{"role":"user","content":prompt}],
        temperature=0,
        max_tokens=64,
    ).choices[0].message.content

def llama_call():
    # pseudo‑call for illustrative purposes
    resp = requests.post(
        "https://api.together.xyz/v1/chat/completions",
        headers={"Authorization": f"Bearer {os.getenv('TOGETHER_API_KEY')}" },
        json={
            "model": "meta-llama/Llama-3-8b-instruct",
            "messages": [{"role":"user","content": prompt}],
            "temperature": 0,
            "max_tokens": 64,
        },
    ).json()
    return resp["choices"][0]["message"]["content"]

# Parse & normalise outputs
def normalise(raw:str) -> list:
    try:
        return json.loads(raw)
    except Exception:
        # Fallback: naive bullet parser
        return [ln.strip("-• ") for ln in raw.splitlines() if ln.strip()]

for name, fn in [("GPT‑4o", gpt_call), ("Llama‑3‑8B", llama_call)]:
    out = normalise(fn())
    print(name, "→", out)



**Cheat‑sheet:**  
| Provider | Temperature≈ | Stop tokens |
|----------|--------------|-------------|
| OpenAI   | 0‑0.2 for determinism | `\nEND` |
| Together | 0‑0.15 | `</s>` |

Adjust these to keep outputs structurally aligned.
